In [2]:
import sqlite3
from datetime import datetime, timedelta
import random

# Initialize Tables

In [ ]:
# Initialize Database
def initialize_database():
    # Connect to SQLite database
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    # Create a table for users
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_dates (
            date STRING PRIMARY KEY,
            year STRING NOT NULL,
            month STRING NOT NULL,
            day STRING NOT NULL,
            is_holiday STRING NOT NULL,
            is_weekend STRING NOT NULL,  
            FOREIGN KEY (date) REFERENCES fact_orders (date)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_restaurants (
            restaurant_id INTEGER PRIMARY KEY AUTOINCREMENT,
            restaurant_name STRING NOT NULL,
            address STRING NOT NULL,
            cuisine STRING NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_menu (
                menu_id INTEGER PRIMARY KEY AUTOINCREMENT,
                restaurant_id INTEGER NOT NULL,
                cost INTEGER NOT NULL,
                prep_time INTEGER NOT NULL,
                FOREIGN KEY(restaurant_id) REFERENCES dim_restaurants(restaurant_id)
        )           
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_customers (
            customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_name STRING NOT NULL,
            customer_email STRING NOT NULL,
            customer_phone STRING NOT NULL,
            customer_address STRING NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_drivers (
            driver_id INTEGER PRIMARY KEY AUTOINCREMENT,
            driver_name STRING NOT NULL,
            driver_email STRING NOT NULL,
            driver_phone STRING NOT NULL,
            driver_address STRING NOT NULL
        )
    ''')       

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS fact_orders (
            order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER NOT NULL,
            restaurant_id INTEGER NOT NULL,
            menu_id INTEGER NOT NULL,
            driver_id INTEGER NOT NULL,
            order_date STRING NOT NULL,
            order_time STRING NOT NULL,
            prep_time INTEGER NOT NULL,
            quantity INTEGER NOT NULL,
            driver_location STRING NOT NULL,
            pickup_location STRING NOT NULL,
            estimated_pick_up_time STRING NOT NULL,
            pick_up_time STRING NOT NULL,
            delivery_location STRING NOT NULL,
            estimated_delivery_time STRING NOT NULL,
            delivery_time STRING NOT NULL,
            sub_total_cost INTEGER NOT NULL,
            tip INTEGER NOT NULL,
            total_cost INTEGER NOT NULL,
            simulation_id INTEGER NOT NULL,
            version_id INTEGER NOT NULL default 1,
            FOREIGN KEY(customer_id) REFERENCES dim_customers(customer_id),
            FOREIGN KEY(restaurant_id) REFERENCES dim_restaurants(restaurant_id),
            FOREIGN KEY(menu_id) REFERENCES dim_menu(menu_id),
            FOREIGN KEY(driver_id) REFERENCES dim_drivers(driver_id),
            FOREIGN KEY(order_date) REFERENCES dim_dates(date)
        )
    ''')

    
    conn.commit()
    conn.close()
    print("Database initialized and tables created.")
initialize_database()

Database initialized and tables created.


# Populate Tables

## `dim_dates`

In [23]:
# Populate Database
def populate_dim_dates():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    # Define Date Range
    start_date = datetime.strptime('2025-01-01', '%Y-%m-%d')  
    end_date = datetime.strptime('2025-12-31', '%Y-%m-%d')


    holidays = [
        datetime.strptime('2025-01-01', '%Y-%m-%d'),  # New Year's Day
        datetime.strptime('2025-01-20', '%Y-%m-%d'),  # Martin Luther King Jr. Day
        datetime.strptime('2025-02-17', '%Y-%m-%d'),  # Presidents' Day
        datetime.strptime('2025-05-26', '%Y-%m-%d'),  # Memorial Day
        datetime.strptime('2025-06-19', '%Y-%m-%d'),  # Juneteenth
        datetime.strptime('2025-07-04', '%Y-%m-%d'),  # Independence Day
        datetime.strptime('2025-09-01', '%Y-%m-%d'),  # Labor Day
        datetime.strptime('2025-10-13', '%Y-%m-%d'),  # Columbus Day
        datetime.strptime('2025-11-11', '%Y-%m-%d'),  # Veterans Day
        datetime.strptime('2025-11-27', '%Y-%m-%d'),  # Thanksgiving
        datetime.strptime('2025-12-25', '%Y-%m-%d'),  # Christmas
    ]

    # Iterate and Populate
    current_date = start_date
    while current_date <= end_date:
        date = current_date.strftime('%Y-%m-%d')
        year = current_date.year
        month = current_date.month
        day = current_date.day
        is_holiday = 'Y' if current_date in holidays else 'N'
        is_weekend = 'Y' if current_date.weekday() >= 5 else 'N'

        cursor.execute('''
            INSERT INTO dim_dates (date, year, month, day, is_holiday, is_weekend)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (date, year, month, day, is_holiday, is_weekend))
        
        current_date += timedelta(days=1)

    conn.commit()
    conn.close()
    print("dim_dates table populated with date range from {} to {}.".format(start_date, end_date))
#populate_dim_dates()

# Query the Database
def query_dim_dates():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM dim_dates LIMIT 10;')
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()
query_dim_dates()

('2025-01-01', 2025, 1, 1, 'Y', 'N')
('2025-01-02', 2025, 1, 2, 'N', 'N')
('2025-01-03', 2025, 1, 3, 'N', 'N')
('2025-01-04', 2025, 1, 4, 'N', 'Y')
('2025-01-05', 2025, 1, 5, 'N', 'Y')
('2025-01-06', 2025, 1, 6, 'N', 'N')
('2025-01-07', 2025, 1, 7, 'N', 'N')
('2025-01-08', 2025, 1, 8, 'N', 'N')
('2025-01-09', 2025, 1, 9, 'N', 'N')
('2025-01-10', 2025, 1, 10, 'N', 'N')


## `dim_restaurants`

In [ ]:
def generate_restaurant_names(num_names=10):
    adjectives = [
        "Tasty", "Delicious", "Savory", "Spicy", "Sweet", "Zesty", "Hearty", "Fresh", "Crispy", "Golden",
        "Fiery", "Bold", "Tangy", "Juicy", "Smoky", "Rich", "Creamy", "Lively", "Exotic", "Rustic",
        "Elegant", "Classic", "Modern", "Cozy", "Charming", "Vibrant", "Wholesome", "Artisan", "Fusion", "Gourmet",
        "Homestyle", "Authentic", "Fluffy", "Cheesy", "Crunchy", "Sizzling", "Warm", "Heavenly", "Bold", "Earthy"
    ]

    nouns = [
        "Bistro", "Grill", "Kitchen", "Cafe", "Diner", "Eatery", "Bar", "Lounge", "Bakery", "Tavern",
        "Pizzeria", "Smokehouse", "Steakhouse", "Cantina", "Pub", "Brasserie", "Trattoria", "Ristorante", "Chophouse", "Buffet",
        "Canteen", "Patio", "Terrace", "Inn", "Taproom", "Delicatessen", "Food Hall", "Oven", "Pantry", "Cellar",
        "Garden", "Table", "Spot", "Corner", "Place", "House", "Shack", "Joint", "Depot", "Parlor"
    ]

    restaurant_names = []
    for _ in range(num_names):
        name = f"{random.choice(adjectives)} {random.choice(nouns)}"
        restaurant_names.append(name)

    return restaurant_names



def populate_dim_restaurants():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()
    
    cuisines = ["Italian", "Chinese", "Mexican", "Indian", "American", "French", "Japanese", "Thai", "Greek", "Mediterranean"]
    restaurant_names = generate_restaurant_names(20)
    
    for name in restaurant_names:
        address = f"{random.randint(1, 100),random.randint(1, 100)}"
        cuisine = random.choice(cuisines)
        #print(name, address, cuisine)

        cursor.execute('''
            INSERT INTO dim_restaurants (restaurant_name, address, cuisine)
            VALUES (?, ?, ?)
        ''', (name, address, cuisine))

    conn.commit()
    conn.close()
    print("dim_restaurants table populated with random data.")

# populate_dim_restaurants()


def query_dim_restaurants():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM dim_restaurants;')
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()
query_dim_restaurants()


dim_restaurants table populated with random data.
(1, 'Fresh Bistro', '(87, 96)', 'Japanese')
(2, 'Heavenly Ristorante', '(66, 92)', 'French')
(3, 'Crunchy Taproom', '(9, 47)', 'Thai')
(4, 'Earthy Pizzeria', '(57, 90)', 'Greek')
(5, 'Hearty Delicatessen', '(56, 75)', 'Mediterranean')
(6, 'Cozy Terrace', '(23, 70)', 'Chinese')
(7, 'Hearty Bistro', '(24, 20)', 'Thai')
(8, 'Exotic Patio', '(28, 57)', 'French')
(9, 'Modern Cellar', '(78, 64)', 'Chinese')
(10, 'Artisan Pub', '(49, 72)', 'Japanese')
(11, 'Artisan Lounge', '(89, 32)', 'Thai')
(12, 'Creamy Buffet', '(92, 57)', 'Indian')
(13, 'Cheesy Canteen', '(35, 47)', 'Italian')
(14, 'Sizzling Terrace', '(21, 5)', 'Chinese')
(15, 'Smoky Spot', '(76, 48)', 'Indian')
(16, 'Crunchy Bakery', '(35, 50)', 'Mexican')
(17, 'Elegant Oven', '(52, 75)', 'Greek')
(18, 'Elegant Cellar', '(3, 82)', 'Indian')
(19, 'Cheesy Oven', '(91, 5)', 'Greek')
(20, 'Cheesy Taproom', '(43, 37)', 'American')


## `dim_menu`

In [ ]:
# This table is simpler, it will just have 5 items per restaurant, a random cost and prep time. Cost will be between 5 and 50, prep time between 10 and 60 minutes. Normally distributed.
def populate_dim_menu():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    # Get all restaurant IDs
    cursor.execute('SELECT restaurant_id FROM dim_restaurants;')
    restaurant_ids = [row[0] for row in cursor.fetchall()]

    for restaurant_id in restaurant_ids:
        for _ in range(5):  # 5 items per restaurant
            cost = random.randint(5, 30)  # Random cost between 5 and 50
            prep_time = random.randint(5, 45)  # Random prep time between 10 and 60 minutes
            #print(restaurant_id, cost, prep_time)
            cursor.execute('''
                INSERT INTO dim_menu (restaurant_id, cost, prep_time)
                VALUES (?, ?, ?)
            ''', (restaurant_id, cost, prep_time))
            

    conn.commit()
    conn.close()
    print("dim_menu table populated with random data.")
# populate_dim_menu()


def query_dim_menu():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM dim_menu;')
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()
query_dim_menu()

dim_menu table populated with random data.
(1, 1, 13, 16)
(2, 1, 8, 32)
(3, 1, 28, 11)
(4, 1, 28, 9)
(5, 1, 26, 31)
(6, 2, 6, 14)
(7, 2, 18, 41)
(8, 2, 6, 26)
(9, 2, 11, 22)
(10, 2, 18, 10)
(11, 3, 28, 33)
(12, 3, 27, 43)
(13, 3, 14, 21)
(14, 3, 14, 41)
(15, 3, 7, 44)
(16, 4, 5, 25)
(17, 4, 28, 26)
(18, 4, 7, 16)
(19, 4, 21, 8)
(20, 4, 22, 44)
(21, 5, 18, 12)
(22, 5, 13, 17)
(23, 5, 13, 34)
(24, 5, 15, 15)
(25, 5, 25, 43)
(26, 6, 10, 26)
(27, 6, 16, 24)
(28, 6, 22, 25)
(29, 6, 13, 18)
(30, 6, 27, 12)
(31, 7, 9, 19)
(32, 7, 6, 20)
(33, 7, 14, 41)
(34, 7, 23, 33)
(35, 7, 10, 39)
(36, 8, 21, 43)
(37, 8, 24, 8)
(38, 8, 17, 32)
(39, 8, 16, 14)
(40, 8, 30, 41)
(41, 9, 25, 22)
(42, 9, 12, 45)
(43, 9, 7, 40)
(44, 9, 9, 18)
(45, 9, 8, 39)
(46, 10, 26, 11)
(47, 10, 25, 14)
(48, 10, 16, 24)
(49, 10, 24, 21)
(50, 10, 7, 8)
(51, 11, 11, 44)
(52, 11, 24, 28)
(53, 11, 29, 14)
(54, 11, 14, 13)
(55, 11, 19, 45)
(56, 12, 28, 23)
(57, 12, 9, 30)
(58, 12, 20, 39)
(59, 12, 25, 16)
(60, 12, 14, 23)
(61, 13,

## `dim_customers`

In [54]:
def generate_names(num_names=10):
    first_names = [
        "John", "Jane", "Alice", "Bob", "Charlie", "Diana", "Eve", "Frank", "Grace", "Hank",
        "Ivy", "Jack", "Kathy", "Leo", "Mia", "Nina", "Oscar", "Paul", "Quinn", "Rita",
        "Sam", "Tina", "Uma", "Vera", "Will", "Xena", "Yara", "Zane", "Aaron", "Bella",
        "Cathy", "David", "Ella", "Fiona", "George", "Holly", "Ian", "Jasmine", "Kevin", "Liam",
        "Mason", "Nora", "Oliver", "Penny", "Quincy", "Riley", "Sophie", "Tom", "Ursula", "Victor"
        "Wendy", "Xander", "Yasmine", "Zoe", "Adam", "Becky", "Cameron", "Derek", "Elena",
        "Felix", "Gina", "Henry", "Isla", "Jake", "Kylie", "Leo", "Maya", "Nate", "Opal",
    ]

    last_names = [
        "Smith", "Johnson", "Williams", "Jones", "Brown",
        "Davis", "Miller", "Wilson", "Moore", "Taylor",
        "Anderson", "Thomas", "Jackson", "White",
        "Harris", "Martin", "Thompson", "Garcia",
        "Martinez", "Robinson", "Clark", "Rodriguez", 
        "Lewis", "Lee", "Walker", "Hall", "Allen",
        "Young", "Hernandez", "King", "Wright", "Lopez",
        "Hill", "Scott", "Green", "Adams", "Baker",
        "Gonzalez", "Nelson", "Carter", "Mitchell",
        "Perez", "Roberts", "Turner", "Phillips",
    ]

    fake_names = []
    for _ in range(num_names):
        name = f"{random.choice(first_names)} {random.choice(last_names)}"
        fake_names.append(name)

    return fake_names

def populate_dim_customers():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    customer_names = generate_names(100)

    for name in customer_names:
        email = f"{name.replace(' ', '.').lower()}@email.com"
        phone = f"+1-{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        address = f"{random.randint(1, 100),random.randint(1, 100)}"
        cursor.execute('''
            INSERT INTO dim_customers (customer_name, customer_email, customer_phone, customer_address)
            VALUES (?, ?, ?, ?)
        ''', (name, email, phone, address))
        

    conn.commit()
    conn.close()
    print("dim_customers table populated with random data.")
# populate_dim_customers()

def query_dim_customers():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM dim_customers;')
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()
query_dim_customers()

(1, 'Nora Hall', 'nora.hall@email.com', '+1-721-189-3310', '(63, 97)')
(2, 'Xander King', 'xander.king@email.com', '+1-704-894-8857', '(51, 23)')
(3, 'Jasmine Martin', 'jasmine.martin@email.com', '+1-297-992-4826', '(35, 53)')
(4, 'Liam Garcia', 'liam.garcia@email.com', '+1-219-599-3487', '(29, 60)')
(5, 'Becky Jones', 'becky.jones@email.com', '+1-269-205-4232', '(29, 5)')
(6, 'Zane Moore', 'zane.moore@email.com', '+1-203-269-8191', '(51, 22)')
(7, 'Tina Allen', 'tina.allen@email.com', '+1-993-695-6338', '(82, 78)')
(8, 'Oliver Baker', 'oliver.baker@email.com', '+1-292-279-1380', '(52, 91)')
(9, 'Kathy Moore', 'kathy.moore@email.com', '+1-581-745-6577', '(42, 13)')
(10, 'Nora Lopez', 'nora.lopez@email.com', '+1-694-339-9072', '(35, 8)')
(11, 'Nate Phillips', 'nate.phillips@email.com', '+1-317-175-1047', '(22, 9)')
(12, 'Xena Wilson', 'xena.wilson@email.com', '+1-537-161-7200', '(8, 94)')
(13, 'Xena Rodriguez', 'xena.rodriguez@email.com', '+1-167-250-6288', '(50, 84)')
(14, 'George Jone

## `dim_drivers`

In [ ]:

def populate_dim_drivers():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    driver_names = generate_names(20)

    for name in driver_names:
        email = f"{name.replace(' ', '.').lower()}@email.com"
        phone = f"+1-{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        address = f"{random.randint(1, 100),random.randint(1, 100)}"
        # print(name, email, phone, address)
        cursor.execute('''
            INSERT INTO dim_drivers (driver_name, driver_email, driver_phone, driver_address)
            VALUES (?, ?, ?, ?)
        ''', (name, email, phone, address))

    conn.commit()
    conn.close()
    print("dim_drivers table populated with random data.")
# populate_dim_drivers()

def query_dim_drivers():
    conn = sqlite3.connect('delivery_app.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM dim_drivers;')
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()
query_dim_drivers()

dim_drivers table populated with random data.
(1, 'Jasmine Wilson', 'jasmine.wilson@email.com', '+1-624-163-1047', '(78, 75)')
(2, 'Zane Johnson', 'zane.johnson@email.com', '+1-616-715-9787', '(8, 95)')
(3, 'Bob Harris', 'bob.harris@email.com', '+1-950-502-2582', '(27, 22)')
(4, 'Elena Johnson', 'elena.johnson@email.com', '+1-579-998-2761', '(24, 18)')
(5, 'Bob Allen', 'bob.allen@email.com', '+1-313-964-8541', '(95, 73)')
(6, 'Cameron Robinson', 'cameron.robinson@email.com', '+1-866-467-4681', '(79, 70)')
(7, 'Becky Brown', 'becky.brown@email.com', '+1-555-312-4699', '(65, 61)')
(8, 'VictorWendy Turner', 'victorwendy.turner@email.com', '+1-488-911-4627', '(51, 56)')
(9, 'Opal Smith', 'opal.smith@email.com', '+1-789-949-1977', '(94, 11)')
(10, 'Charlie Phillips', 'charlie.phillips@email.com', '+1-998-898-6310', '(85, 10)')
(11, 'Gina Rodriguez', 'gina.rodriguez@email.com', '+1-465-451-4134', '(28, 57)')
(12, 'Aaron Smith', 'aaron.smith@email.com', '+1-105-315-7113', '(27, 81)')
(13, 'Vi